### Naive Bayes朴素贝叶斯分类器


#### ·贝叶斯公式

$P(C_{i}|x)=\frac{P(C_{i})P(x|C_{i})}{P(x)}$

其中：$x$为自变量

·$C_{i}$为第i个类别

·$P(C_{i}|x)$为满足$x$时，其属于$C_{i}$的后验概率

·$P(C_{i})$为属于$C_{i}$类别的先验概率

·$P(x|C_{i})$为在类别$C_{i}$中是$x$的概率

·朴素贝叶斯分类器假设各个特征之间相互独立，即给定类别的情况下，各个特征发生的概率之间相互独立。因此，可以将上述公式简化为：$P(C_i|\textbf{x}) = P(C_i) \cdot \prod_{j=1}^{n} P(x_j|C_i)$

In [80]:
import numpy  as np
import pandas as pd
data = [
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        [ 'Sunny', 'Hot', 'High', 'Strong', 'No'],
        [ 'Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        [ 'Rain', 'Mild', 'High', 'Weak', 'Yes'],
        [ 'Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        [ 'Rain', 'Cool', 'Normal', 'Strong', 'No'],
        [ 'Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        [ 'Sunny', 'Mild', 'High', 'Weak', 'No'],
        [ 'Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        [ 'Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        [ 'Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        [ 'Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        [ 'Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
        [ 'Rain', 'Mild', 'High', 'Strong', 'No'],
        ]

In [81]:
Data = pd.DataFrame(data,columns=['x1','x2','x3','x4','y'])
Data.head()

,x1,x2,x3,x4,y
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [82]:
cols=Data.shape[1]  #5列，4特征+1标签
X_data=Data.iloc[:,:cols-1]
y_data=Data.iloc[:,cols-1:]
featureNames=X_data.columns
print(featureNames)

Index(['x1', 'x2', 'x3', 'x4'], dtype='object')


·模型训练

In [83]:

def Naive_Bayes(X_data,y_data):

    #---------第一步：计算先验概率
    x=X_data.values
    y=y_data.values
    y_unique = np.unique(y)#将y中重复的元素删除，即提取出类别数组
    prior_prob=np.zeros(len(y_unique))#存放长度为标签个数的先验概率
    #计算所有类别的先验概率
    for i in range(len(y_unique)):
        prior_prob[i] = (y == y_unique[i]).sum() / len(y)
    #---------第二步：计算条件概率概率/似然
    condition_prob={}           #是一个空字典，用于存储每个特征下的条件概率
    for feat in featureNames:   #根据特征名来遍历每一个特征
        x_unique=list(set(X_data[feat]))#该特征下所有唯一取值的列表
        x_condition_prob=np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j, k] = ((X_data[feat] == x_unique[k]) & (y_data.y == y_unique[j])).sum() / (y == y_unique[j]).sum()
        x_condition_prob=pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat]=x_condition_prob
    return prior_prob,condition_prob
    

In [84]:
prior_prob,condition_prob=Naive_Bayes(X_data,y_data)
print(prior_prob)
print(condition_prob['x1'])
print(condition_prob['x2'])
print(condition_prob['x3'])
print(condition_prob['x4'])


[0.35714286 0.64285714]
         Rain     Sunny  Overcast
No   0.400000  0.600000  0.000000
Yes  0.333333  0.222222  0.444444
         Cool       Hot      Mild
No   0.200000  0.400000  0.400000
Yes  0.333333  0.222222  0.444444
         High    Normal
No   0.800000  0.200000
Yes  0.333333  0.666667
         Weak    Strong
No   0.400000  0.600000
Yes  0.666667  0.333333


朴素贝叶斯分类器的“朴素”假设是特征之间相互独立。这意味着每个特征$X_{i}$贡献的信息独立于其他特征。因此，我们可以将联合似然 $P(X|C)$分解为各个特征的似然的乘积：

$P(X|C)=P(X_{1}|C)·P(X_{2}|C)...P(X_{n}|C)·$

**Prediction的基本步骤如下：**

·初始化

·遍历每个样本和类别

·计算条件概率

·概率累乘

·归一化处理

·存储后验概率

In [85]:
#注意测试数据是dataFrame格式的
def Prediction(testData, prior, condition_prob):
    numClass = prior.shape[0]  # 类别数量
    featureNames = testData.columns  # 特征名列表
    numSample = testData.shape[0]  # 测试样本数
    post_prob = np.zeros((numSample, numClass))  # 存储每个样本的后验概率

    for k in range(numSample):  # 遍历每个测试样本
        prob_k = np.zeros((numClass,))  # 存储当前样本每个类别的概率
        for i in range(numClass):  # 遍历每个类别
            if i == 0:
                y = 'No'
            if i == 1:
                y = 'Yes'
            pri = prior[i]  # 获取当前类别的先验概率 P(C)

            for feat in featureNames:  # 遍历每个特征
                feat_val = testData[feat][k]  # 当前样本的特征值
                cp = condition_prob[feat]  # 获取当前特征的条件概率表
                cp_val = cp.loc[y, feat_val]  # 当前特征和值在当前类别下的条件概率 P(X|C)
                pri *= cp_val  # 累乘似然

            prob_k[i] = pri  # 计算完所有特征后，存储当前类别的非归一化后验概率

        prob = prob_k / np.sum(prob_k, axis=0)  # 归一化，确保概率和为1
        post_prob[k, :] = prob  # 将后验概率存储到数组中

    return post_prob

In [86]:
test_data=[['Sunny','Cool','High','Strong']]
testdata=pd.DataFrame(test_data,columns=[ 'x1', 'x2', 'x3', 'x4'])
testdata.head()

,x1,x2,x3,x4
0,Sunny,Cool,High,Strong


In [87]:
postPrior=Prediction(testdata,prior_prob,condition_prob)
postPrior

array([[0.79541735, 0.20458265]])

In [88]:
if (postPrior[0][0]>postPrior[0][1]):
    print('No')

else:
    print('Yes')


No
